In [1]:
import pandas as pd
from IPython.display import display, HTML
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

Using TensorFlow backend.


In [2]:
# For plot

def prepare_standardplot(title, xlabel):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title)
    ax1.set_ylabel('categorical cross entropy')
    ax1.set_xlabel(xlabel)
    ax1.set_yscale('log')
    ax2.set_ylabel('accuracy [% correct]')
    ax2.set_xlabel(xlabel)
    return fig, ax1, ax2

def finalize_standardplot(fig, ax1, ax2):
    ax1handles, ax1labels = ax1.get_legend_handles_labels()
    if len(ax1labels) > 0:
        ax1.legend(ax1handles, ax1labels)
    ax2handles, ax2labels = ax2.get_legend_handles_labels()
    if len(ax2labels) > 0:
        ax2.legend(ax2handles, ax2labels)
    fig.tight_layout()
    plt.subplots_adjust(top=0.9)

def plot_history(history, title):
    fig, ax1, ax2 = prepare_standardplot(title, 'epoch')
    ax1.plot(history.history['loss'], label = "training")
    ax1.plot(history.history['val_loss'], label = "validation")
    ax2.plot(history.history['binary_accuracy'], label = "training")
    ax2.plot(history.history['val_binary_accuracy'], label = "validation")
    finalize_standardplot(fig, ax1, ax2)
    return fig

In [3]:
df = pd.read_csv('Data/bitstampUSD_1-min_data_2012-01-01_to_2018-01-08.csv')

In [4]:
df= df.drop('Timestamp',1)
df[['Open','High','Low','Close']] = df[['Open','High','Low','Close']].apply(lambda x: np.log(x))
df = (df-df.mean())/df.std()

In [5]:
df.describe()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
count,3.161057e+06,3.161057e+06,3.161057e+06,3.161057e+06,3.161057e+06,3.161057e+06,3.161057e+06
mean,3.015734e-13,4.370263e-13,3.686399e-13,6.005367e-13,-5.074202e-14,8.689802e-16,-3.917382e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.150525e+00,-2.150622e+00,-2.634462e+00,-2.634494e+00,-3.099179e-01,-1.844678e-01,-4.246458e-01
25%,-3.932670e-01,-3.934676e-01,-3.933023e-01,-3.932387e-01,-2.974788e-01,-1.832231e-01,-3.772309e-01
50%,2.458171e-01,2.458036e-01,2.457736e-01,2.458136e-01,-2.521185e-01,-1.776162e-01,-2.588227e-01
75%,5.338638e-01,5.340192e-01,5.338080e-01,5.339054e-01,-6.396828e-02,-1.345711e-01,-1.350566e-01
max,2.302785e+00,2.302313e+00,2.302864e+00,2.302737e+00,1.626979e+02,9.636451e+01,8.272274e+00


In [6]:
def add_MA_n_days_age(num_days):
    num_days_str = str(num_days)
    
    df[['Open_W_MA_'+num_days_str,'High_W_MA_'+num_days_str,'Low_W_MA_'+num_days_str,'Close_W_MA_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).mean()
    df[['Open_MA_'+num_days_str,'High_MA_'+num_days_str,'Low_MA_'+num_days_str,'Close_MA_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).mean()
    df[['Open_EMA_'+num_days_str,'High_EMA_'+num_days_str,'Low_EMA_'+num_days_str,'Close_EMA_'+num_days_str]] = df[['Open','High','Low','Close']].ewm(span=day_window * num_days).mean()

    df[['Open_MAX_'+num_days_str,'High_MAX_'+num_days_str,'Low_MAX_'+num_days_str,'Close_MAX_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).max()
    df[['Open_MIN_'+num_days_str,'High_MIN_'+num_days_str,'Low_MIN_'+num_days_str,'Close_MIN_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).min()

    df['Open_TENKAN_'+num_days_str] = 0.5 * (df['Open_MIN_'+num_days_str] + df['Open_MAX_'+num_days_str])
    df['High_TENKAN_'+num_days_str] = 0.5 * (df['High_MIN_'+num_days_str] + df['High_MAX_'+num_days_str])
    df['Low_TENKAN_'+num_days_str] = 0.5 * (df['Low_MIN_'+num_days_str] + df['Low_MAX_'+num_days_str])
    df['Close_TENKAN_'+num_days_str] = 0.5 * (df['Close_MIN_'+num_days_str] + df['Close_MAX_'+num_days_str])

def add_prices_n_days_ago(num_days):
    num_days_str = str(num_days)
    df[['Open_S_'+num_days_str,'High_S_'+num_days_str,'Low_S_'+num_days_str,'Close_S_'+num_days_str]] = df[['Open','High','Low','Close']].shift(day_window * num_days)

In [7]:
day_window = 60 * 24

df[['Open_W','High_W','Low_W','Close_W']] = df[['Open','High','Low','Close']].divide(df['Volume_(BTC)'],axis = 0)

add_prices_n_days_ago(1)
add_prices_n_days_ago(2)
add_prices_n_days_ago(3)

add_MA_n_days_age(5)
add_MA_n_days_age(10)
add_MA_n_days_age(15)

plusieurs y gains selon différents temps + vote

In [8]:
mean = (df['Open'] + df['Close']) * 0.5
res = mean.shift(day_window)

df['y'] = ((mean - res) > 0).astype(int)

In [9]:
display(df.sample(10))

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Open_W,High_W,Low_W,...,Close_MAX_15,Open_MIN_15,High_MIN_15,Low_MIN_15,Close_MIN_15,Open_TENKAN_15,High_TENKAN_15,Low_TENKAN_15,Close_TENKAN_15,y
28593,-1.893913,-1.894032,-1.893733,-1.893842,0.501800,-0.181067,-0.423575,-3.774235,-3.774472,-3.773878,...,-1.804793,-1.956087,-1.956200,-1.955912,-1.956014,-1.880474,-1.880594,-1.880294,-1.880403,0
2278155,0.332510,0.332200,0.331751,0.332296,-0.247487,-0.165670,-0.228561,-1.343544,-1.342293,-1.340477,...,0.358198,0.313370,0.313123,0.313657,0.313378,0.335775,0.335584,0.336005,0.335788,0
1855185,0.104927,0.104799,0.105258,0.105103,-0.257139,-0.174195,-0.298483,-0.408054,-0.407557,-0.409342,...,0.153242,0.034635,0.034371,0.034898,0.034630,0.093990,0.094473,0.093919,0.093936,0
1975332,0.018460,0.019898,0.018209,0.018455,6.413254,0.924690,-0.317968,0.002878,0.003103,0.002839,...,0.025830,-0.017528,-0.017808,-0.018051,-0.017898,0.004048,0.003861,0.003939,0.003966,1
2799366,0.929671,0.929314,0.930065,0.929666,-0.301252,-0.176250,0.196505,-3.086028,-3.084843,-3.087335,...,0.935945,0.829257,0.828905,0.829643,0.829251,0.882233,0.882249,0.882266,0.882598,1
681392,-0.549072,-0.544833,-0.550629,-0.544942,0.923724,-0.116075,-0.389913,-0.594411,-0.589822,-0.596097,...,0.048714,-0.810614,-0.808634,-0.863136,-0.808486,-0.380958,-0.380111,-0.407055,-0.379886,1
2075807,0.343181,0.342871,0.343531,0.343189,-0.285591,-0.176988,-0.224373,-1.201655,-1.200568,-1.202877,...,0.355732,0.202790,0.203550,0.203128,0.202906,0.279285,0.279565,0.279602,0.279319,1
1511248,0.272267,0.272015,0.272558,0.272329,-0.255896,-0.169973,-0.250092,-1.063974,-1.062989,-1.065113,...,0.340215,0.155737,0.156243,0.152939,0.154571,0.247972,0.248070,0.246748,0.247393,0
1248788,0.333894,0.333584,0.334242,0.333901,-0.260675,-0.169594,-0.227943,-1.280882,-1.279694,-1.282219,...,0.353019,0.301013,0.300928,0.300443,0.300105,0.326889,0.327184,0.326723,0.326562,0
644555,-0.633232,-0.633459,-0.633032,-0.633271,0.159043,-0.162358,-0.395360,-3.981525,-3.982952,-3.980264,...,-0.584249,-0.974272,-0.974470,-0.979348,-0.979552,-0.779277,-0.779492,-0.781676,-0.781900,0


In [10]:
df = df.dropna(how ='any')

y = df.y
df = (df-df.mean())/df.std()
df.y = y
display(df.describe())

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Open_W,High_W,Low_W,...,Close_MAX_15,Open_MIN_15,High_MIN_15,Low_MIN_15,Close_MIN_15,Open_TENKAN_15,High_TENKAN_15,Low_TENKAN_15,Close_TENKAN_15,y
count,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,...,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06,3.139458e+06
mean,-3.392586e-13,-5.906626e-13,5.790729e-14,-4.446534e-13,6.607257e-14,1.272720e-14,-7.354502e-14,-2.902058e-17,-1.760952e-16,1.528909e-16,...,-3.545786e-13,-2.427295e-12,7.563479e-14,4.981674e-13,-7.594128e-13,1.256374e-12,-4.448784e-12,-3.648817e-12,-1.063144e-12,5.553561e-01
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.969263e-01
min,-2.183942e+00,-2.184021e+00,-2.672430e+00,-2.672439e+00,-3.100156e-01,-1.851188e-01,-4.263604e-01,-3.251624e+02,-3.250301e+02,-3.252853e+02,...,-2.080054e+00,-2.102651e+00,-2.103383e+00,-2.514825e+00,-2.516158e+00,-2.063340e+00,-2.064458e+00,-2.045646e+00,-2.046634e+00,0.000000e+00
25%,-3.881069e-01,-3.878531e-01,-3.884049e-01,-3.882137e-01,-2.977100e-01,-1.838289e-01,-3.769825e-01,-1.635690e-03,-1.634168e-03,-1.635967e-03,...,-3.817015e-01,-4.439232e-01,-4.454905e-01,-4.261698e-01,-4.277904e-01,-4.057692e-01,-4.057031e-01,-4.081473e-01,-4.090605e-01,0.000000e+00
50%,2.377383e-01,2.377454e-01,2.376952e-01,2.377903e-01,-2.522784e-01,-1.781013e-01,-2.600888e-01,1.490599e-04,1.501120e-04,1.486858e-04,...,2.438378e-01,2.741989e-01,2.751972e-01,2.802574e-01,2.797808e-01,2.623724e-01,2.623112e-01,2.668872e-01,2.659238e-01,1.000000e+00
75%,5.276694e-01,5.277287e-01,5.276388e-01,5.276713e-01,-6.478471e-02,-1.345290e-01,-1.364458e-01,2.967752e-03,2.967989e-03,2.967562e-03,...,5.382577e-01,5.675958e-01,5.660385e-01,5.780274e-01,5.762685e-01,5.394798e-01,5.385775e-01,5.390893e-01,5.383868e-01,1.000000e+00
max,2.311080e+00,2.310591e+00,2.311170e+00,2.311028e+00,1.621828e+02,9.604464e+01,8.246152e+00,1.272633e+03,1.272879e+03,1.272564e+03,...,2.253494e+00,2.193020e+00,2.191008e+00,2.164114e+00,2.165990e+00,2.227555e+00,2.226237e+00,2.231059e+00,2.231849e+00,1.000000e+00


In [11]:
train=df.head(int(df.shape[0]*0.9))
test=df.drop(train.index)

In [ ]:
train_x = train.drop('y',1)
train_y = train['y']
test_x = test.drop('y',1)
test_y = test['y']

model = Sequential()
model.add(Dense(128, input_dim=95, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

history = model.fit(train_x, train_y, epochs = 5,validation_split = 0.1, verbose=1)
scores = model.evaluate(test_x, test_y, verbose=0)

model.save('my_model2.h5')

Train on 2542960 samples, validate on 282552 samples
Epoch 1/5
2542960/2542960 [==============================] - 178s 70us/step - loss: 0.4230 - binary_accuracy: 0.7804 - val_loss: 0.3974 - val_binary_accuracy: 0.7839
Epoch 2/5
2542960/2542960 [==============================] - 171s 67us/step - loss: 0.3254 - binary_accuracy: 0.8448 - val_loss: 0.3223 - val_binary_accuracy: 0.9381
Epoch 3/5
2542960/2542960 [==============================] - 172s 68us/step - loss: 0.2987 - binary_accuracy: 0.8606 - val_loss: 0.2828 - val_binary_accuracy: 0.9465
Epoch 4/5
2542960/2542960 [==============================] - 181s 71us/step - loss: 0.2861 - binary_accuracy: 0.8680 - val_loss: 0.2290 - val_binary_accuracy: 0.9427
Epoch 5/5
2130752/2542960 [========================>.....] - ETA: 28s - loss: 0.2799 - binary_accuracy: 0.8729

In [ ]:
plot_history(history,"test")

In [ ]:
print(scores)

In [ ]:
del model

from keras.models import load_model
model = load_model('my_model.h5')

scores = model.evaluate(test_x, test_y, verbose=0)
print(scores)